In [1]:
import numpy as np
import tensorflow as tf
import cv2

In [2]:
label_list = ['NaN', 'Masked', 'Unmasked']

cap = cv2.VideoCapture(0)
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
cv2.namedWindow("Webcam frame", cv2.WINDOW_AUTOSIZE)
cv2.namedWindow("Detection frame", cv2.WINDOW_AUTOSIZE)

detection_threshold = 0.5
bbox_coord_conversion = np.array([frame_height, frame_width, frame_height, frame_width])

saved_model_dir = "C:\\PATH_TO_SAVED_MODEL_FINED_TUNED\\saved_model"

with tf.Session() as sess:
    metagraph = tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], saved_model_dir)

    input_dict = dict(metagraph.signature_def['serving_default'].inputs)
    output_dict = dict(metagraph.signature_def['serving_default'].outputs)

    input_tensor = sess.graph.get_tensor_by_name(input_dict['input_tensor'].name)
    bounding_boxes = sess.graph.get_tensor_by_name(output_dict['detection_boxes'].name)
    box_scores = sess.graph.get_tensor_by_name(output_dict['detection_scores'].name)
    box_classes = sess.graph.get_tensor_by_name(output_dict['detection_classes'].name)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow("Webcam frame", frame)

        [boxes, scores, classes] = sess.run([bounding_boxes, box_scores, box_classes],
                                            feed_dict={input_tensor: np.expand_dims(frame, axis=0)})

        detections = boxes[scores > detection_threshold]
        detected_classes = classes[scores > detection_threshold]

        for obj in zip(detections, detected_classes):
            bbox_coord = (obj[0] * bbox_coord_conversion).astype(int)
            cv2.rectangle(frame, (bbox_coord[1], bbox_coord[0]), (bbox_coord[3], bbox_coord[2]), (0, 255, 0), 2)
            cv2.putText(img=frame,
                        text=label_list[obj[1].astype(int)],
                        org=(bbox_coord[1], bbox_coord[0]),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=1,
                        color=(0, 255, 0),
                        thickness=2,
                        lineType=2)

        cv2.imshow("Detection frame", frame)

        key = cv2.waitKey(1) & 0xff
        if key == ord('q'):
            cv2.destroyAllWindows()
            break



Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from C:\Users\ExcellentMo3\ActualFiles\DEEP 2022\code\mask_detection\saved_model\variables\variables
